In [2]:
import xml.etree.cElementTree as ET
import bz2
import xml
import sys
import pprint
import pymongo
import re
from collections import defaultdict
from pymongo import MongoClient

In [3]:
seattleOSM = 'seattle_washington.osm'

In [39]:
with open(seattleOSM, 'r') as f:
    
    tagDict = {}
    subTags = {}
    maxLat = 0
    maxLon = 0
    
    for event, elem in ET.iterparse(f):
        if elem.tag in tagDict.keys():
            tagDict[elem.tag]['number'] += 1
            for attribute in elem.attrib:
                if attribute in tagDict[elem.tag].keys():
                    tagDict[elem.tag][attribute]+=1
                else:
                    tagDict[elem.tag][attribute] = 1
        else:
            tagNumber = {'number':1}
            tagDict[elem.tag] = tagNumber
            for attribute in elem.attrib:
                if attribute in tagDict[elem.tag].keys():
                    tagDict[elem.tag][attribute]+=1
                else:
                    tagDict[elem.tag][attribute] = 1
        elem.clear
    pprint.pprint(tagDict)

{'bounds': {'maxlat': 1, 'maxlon': 1, 'minlat': 1, 'minlon': 1, 'number': 1},
 'member': {'number': 40629, 'ref': 40629, 'role': 40629, 'type': 40629},
 'nd': {'number': 7320270, 'ref': 7320270},
 'node': {'changeset': 6620098,
          'id': 6620098,
          'lat': 6620098,
          'lon': 6620098,
          'number': 6620098,
          'timestamp': 6620098,
          'uid': 6619934,
          'user': 6619934,
          'version': 6620098},
 'osm': {'generator': 1, 'number': 1, 'timestamp': 1, 'version': 1},
 'relation': {'changeset': 6057,
              'id': 6057,
              'number': 6057,
              'timestamp': 6057,
              'uid': 6057,
              'user': 6057,
              'version': 6057},
 'tag': {'k': 4295818, 'number': 4295818, 'v': 4295818},
 'way': {'changeset': 632414,
         'id': 632414,
         'number': 632414,
         'timestamp': 632414,
         'uid': 632389,
         'user': 632389,
         'version': 632414}}


In [36]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [37]:
#client = MongoClient('localhost',27017)
#db = client['test-database']
#test_collection = db['seattle']

#clear seattle db before adding data
#db.seattle.remove({})

streetDict
with open(seattleOSM,'r') as f:
    i = 0
    for event,elem in ET.iterparse(f, events=("start",)):
        i+=1
        if i>10000:
            break
        if elem.tag == 'node' or elem.tag == 'way':    
            insertDict={}
            locationList = [0,0]
            tags = {}
            insertDict['type']=elem.tag
            for attributeKey in elem.keys():
                if attributeKey == 'tag':
                    pass
                elif attributeKey == 'lat':

                    locationList[0] = elem.attrib['lat']
                elif attributeKey == 'lon':
                    locationList[1] = elem.attrib['lon']
                else:
                    insertDict[attributeKey] = elem.attrib[attributeKey]
            for tag in elem.iter('tag'):
                try:
                    tagv = tag.attrib['v']
                    tagk = tag.attrib['k']
                    tags[tagk] = tagv
                    if is_street_name(tag):
                        print 'FOUND A STREET NAME------------------------------'
                        print tag.attrib
                except:
                    print 'FOUND A TAG ERROR -----------------'
                    print 'tag.attrib: ',tag.attrib
                    print 'tag.keys(): ',tag.keys()
            #If locationList isn't empty add it to the document
            if locationList != [0,0]:
                insertDict['location']=locationList
            if bool(tags):
                insertDict['tags'] = tags

            
            if 'tags' in insertDict.keys():
                if 'name' in insertDict['tags'].keys():
                    print'\nNew Dict\n'
                    pprint.pprint(insertDict)

            # UNCOMMENT TO WRITE TO DB !!!   
            #test_collection.insert_one(insertDict)
        else:
            pass
        elem.clear()
#db.collection_names(include_system_collections=False)


New Dict

{'changeset': '25678535',
 'id': '25883551',
 'location': ['48.4145385', '-123.3564537'],
 'tags': {'name': 'Cook Street Village', 'place': 'neighbourhood'},
 'timestamp': '2014-09-26T06:03:13Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '5'}

New Dict

{'changeset': '25678535',
 'id': '25903927',
 'location': ['48.4613889', '-123.2970141'],
 'tags': {'name': 'Cadboro Bay Village',
          'place': 'neighbourhood',
          'website': 'http://www.cadborobayvillage.com/'},
 'timestamp': '2014-09-26T06:03:13Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '5'}

New Dict

{'changeset': '14680500',
 'id': '26962238',
 'location': ['48.4601659', '-123.4428921'],
 'tags': {'name': 'Palmer (Fort Victoria)', 'railway': 'station'},
 'timestamp': '2013-01-17T06:49:41Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '3'}

New Dict

{'changeset': '25678535',
 'id': '27047806',
 'location': ['48.4361598', '-123.339766

In [23]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

expectedDirection = ['North','South','East','West','Northwest','Northeast',
                     'Southeast','Southwest']

questionableNames = ['Fir','Pine','Station','Plateau','Broadway','Rise','Meadows']
errorNames = ['Mcgarigle','Mall','James','Tumwater','Dakota','98296']
numberForName = ['214','210']
newNames =['Ridge','Heights','Gate','Terrace','Alley','Way','Loop']
onlyOne = ['Hills','Cove','Landing','Grove']
url =['http://local.safeway.com/wa/tacoma-1594.html']
alreadyChecked = []
investigationList = [questionableNames,errorNames,numberForName,
                     newNames,onlyOne,url]
                     

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave":"Avenue",
            "Rd.":"Road"
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def auditStreetNames(osm_file):
    osm_file = open(osm_file, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                    for typeOfStreet in investigationList:
                        for entry in typeOfStreet:
                            if str(tag.attrib['v'])==str(entry):
                                if entry in alreadyChecked:
                                    pass    
                                else:
                                    print '\n---------------------------------'
                                    print 'ENTRY: ',entry,'\n'
                                    alreadyChecked.append(entry)
                                    pprint.pprint(elem.attrib)
                                    for tag_1 in elem.iter('tag'):
                                        print '\t',tag_1.attrib
                                    print 'Already Checked: ',alreadyChecked
                                
        elem.clear()
    return street_types

def update_name(name, mapping):
    print 'Name: ',name,'\nMapping: ',mapping
    mapKeys = mapping.keys()
    for keys in mapKeys:
        if keys in name:
            name = name[0:name.index(keys)]+mapping[keys]
    return name

def test():
    st_types = auditStreetNames(seattleOSM)
    #pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        print st_type, ' : ',len(ways)
        #for name in ways:
        #    better_name = update_name(name, mapping)
        #    print name, "=>", better_name
        
test()


---------------------------------
ENTRY:  Mcgarigle 

{'changeset': '32996683',
 'id': '356548097',
 'lat': '48.5133762',
 'lon': '-122.2206887',
 'timestamp': '2015-07-31T00:22:40Z',
 'uid': '1529630',
 'user': 'Brad Meteor',
 'version': '3'}
	{'k': 'ele', 'v': '19'}
	{'k': 'name', 'v': 'Evergreen Elementary School'}
	{'k': 'amenity', 'v': 'school'}
	{'k': 'addr:city', 'v': 'Sedro Woolley'}
	{'k': 'addr:street', 'v': 'Mcgarigle'}
	{'k': 'gnis:created', 'v': '10/01/1990'}
	{'k': 'addr:postcode', 'v': '98284'}
	{'k': 'gnis:state_id', 'v': '53'}
	{'k': 'gnis:county_id', 'v': '057'}
	{'k': 'gnis:feature_id', 'v': '1531302'}
	{'k': 'addr:housenumber', 'v': '1111'}
Already Checked:  ['Mcgarigle']

---------------------------------
ENTRY:  James 

{'changeset': '7661586',
 'id': '1216905966',
 'lat': '47.602414',
 'lon': '-122.332316',
 'timestamp': '2011-03-25T01:34:14Z',
 'uid': '337321',
 'user': 'gonzoramos',
 'version': '2'}
	{'k': 'name', 'v': 'Pioneer Square Station'}
	{'k': 'highway

In [14]:
#Print first 100 items in seattle collection

seattle_1 = db.seattle
i=0
for entry in seattle_1.find():
    if i<100:
        pprint.pprint(entry) 
        i+=1
    else:
        break

NameError: name 'db' is not defined